In [1]:
import numpy as np
import pandas as pd

import lightgbm
import matplotlib.pylab as plt
import os

from sklearn.model_selection import train_test_split

Inserting all features from test into one dataframe

In [2]:
folder_train = '../features_train_csv'

features_train = pd.read_csv(os.path.join(folder_train, os.listdir(folder_train)[0]))
for i in range(1,len(os.listdir(folder_train))):
    f = os.path.join(folder_train,os.listdir(folder_train)[i])
    if os.path.isfile(f):
        features_train = pd.concat([features_train, pd.read_csv(f)], axis=1)
print(features_train)
        
        

       average_income_per_grunnkrets  distance_store_busstop  \
0                           561700.0              585.354489   
1                           555720.0              138.798366   
2                           495900.0               41.099519   
3                           521240.0               56.693017   
4                           585360.0              229.077959   
...                              ...                     ...   
12854                       505640.0              150.004125   
12855                       396640.0              322.573926   
12856                       549120.0              174.851316   
12857                       498660.0              121.372663   
12858                       579780.0              189.983143   

       distance_to_competitor  area_grunnkrets  
0                    0.000000         0.155779  
1                   34.547236         0.264278  
2                    0.000000         0.160152  
3                    0.000000      

Repeat for test features

In [3]:
folder_test = '../features_test_csv'

features_test = pd.read_csv(os.path.join(folder_test, os.listdir(folder_test)[0]))
for i in range(1,len(os.listdir(folder_test))):
    f = os.path.join(folder_test,os.listdir(folder_test)[i])
    if os.path.isfile(f):
        features_test = pd.concat([features_test, pd.read_csv(f)], axis=1)
print(features_test)
    

      average_income_per_grunnkrets  distance_store_busstop  \
0                          551600.0               25.630061   
1                          531240.0              111.736189   
2                          523300.0              404.825119   
3                          591380.0              317.141300   
4                          594020.0              101.273354   
...                             ...                     ...   
8572                       560700.0              364.034347   
8573                       593340.0              228.923169   
8574                       629100.0              136.232163   
8575                       590100.0              129.203732   
8576                       529980.0              212.494198   

      distance_to_competitor  area_grunnkrets  
0                  70.085760         0.057027  
1                 177.710439         0.165993  
2                   0.000000         0.236628  
3                 740.150349         0.983436  
4  

Target train data:

In [4]:
target_train = pd.read_csv("../data/stores_train.csv")
target_train = target_train['revenue'].values

#log transform
target_train = np.log1p(target_train)

Light GBM model:

In [5]:
lgbm_model = lightgbm.LGBMRegressor(
    num_leaves=4,
    max_depth=5, 
    random_state=42, 
    silent=True, 
    metric='mse',
    n_jobs=4, 
    n_estimators=1000,
    colsample_bytree=0.95,
    subsample=0.9,
    learning_rate=0.05
)

In [6]:
#print(features_train)
#print(type(features_train))
print(len(features_train))
print(target_train)
print(type(target_train))
target_train = target_train
print(len(target_train))

12859
[17.998 23.828 16.099 ... 38.225  3.642  2.328]
<class 'numpy.ndarray'>
12859


In [7]:
lgbm_model.fit(features_train, target_train)
lgbm_predictions = lgbm_model.predict(features_test)

#Undo log transform
lgbm_predictions = np.expm1(lgbm_predictions)

c:\Python310\lib\site-packages\lightgbm\sklearn.py:598: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via keyword arguments instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "


In [8]:
print(lgbm_predictions)

[ 5.73207311 14.83936583  4.46847189 ... 12.97568844  8.65113663
  8.55905579]


In [9]:
stores_test = pd.read_csv("../data/stores_test.csv")
stores_test_id = stores_test['store_id'].values

In [12]:
predictions = pd.DataFrame()

predictions['id'] = stores_test_id
predictions['predicted'] = lgbm_predictions
predictions.to_csv("../predictions/lightGBM3.csv", index=False)
print(predictions)


                              id  predicted
0     914206820-914239427-717245   5.732073
1     916789157-916823770-824309  14.839366
2       913341082-977479363-2948   4.468472
3      889682582-889697172-28720   7.911306
4     997991699-998006945-417222  13.976502
...                          ...        ...
8572  917323003-917383529-844309   5.792643
8573  917353379-917411824-845904   9.570841
8574  917072302-917089248-833647  12.975688
8575  916960557-916993161-829908   8.651137
8576   987280891-972040746-45320   8.559056

[8577 rows x 2 columns]
